In [60]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import glob
import os, sys
from sklearn import preprocessing
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from ipywidgets import IntProgress



In [49]:
DATA_DIR = '/data2/minki/kaggle/vinbigdata-cxr'

In [50]:
train_df = pd.read_csv(DATA_DIR+'/train.csv')
le = preprocessing.LabelEncoder()  # encode rad_id

train_df['rad_label'] = le.fit_transform(train_df['rad_id'])

train_df.head()


,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,rad_label
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,14
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,1
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,1
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,1


In [51]:
# Make Meta Dict


Error: Pip module Unable to parse debugpy output, please log an issue with https://github.com/microsoft/vscode-jupyter is required for debugging cells. You will need to install it to debug cells.

In [52]:
size_df = pd.read_csv(DATA_DIR+'/png_1024/train_meta.csv')
size_df.head()

,image_id,dim0,dim1
0,000434271f63a053c4128a0ba6352c7f,2836,2336
1,00053190460d56c53cc3e57321387478,2430,1994
2,0005e8e3701dfb1dd93d53e2ff537b6e,3072,3072
3,0006e0a85696f6bb578e84fafa9a5607,3000,3000
4,0007d316f756b3fa0baea2ff514ce945,2880,2304


In [59]:

train_meta_dict = {}

train_ids = train_df['image_id'].unique()
for tid in (train_ids):
    print(tid)
    train_meta_dict[tid] = {}
    
    # get size
    # dicom = pydicom.read_file(DATA_DIR + f'/train/{tid}.dicom')
    # arr =  apply_voi_lut(dicom.pixel_array, dicom)
    # train_meta_dict[tid]['dim0'] = arr.shape[0]
    # train_meta_dict[tid]['dim1'] = arr.shape[1]
    tid_size_row = size_df[size_df['image_id']==tid].iloc[0]
    train_meta_dict[tid]['dim0'] = tid_size_row['dim0']
    train_meta_dict[tid]['dim1'] = tid_size_row['dim1']

    # append bbox
    train_meta_dict[tid]['bbox'] = []
    tid_df = train_df[train_df['image_id'] == tid]
    for ri in range(len(tid_df)):
        row = tid_df.iloc[ri]
        ri_bbox = [row['rad_label'], row['class_name'], row['class_id'], row['x_min'], row['y_min'], row['x_max'], row['y_max']]
        train_meta_dict[tid]['bbox'].append(ri_bbox)

    


50a418190bc3fb1ef1633bf9678929b3
21a10246a5ec7af151081d0cd6d65dc9
9a5094b2563a1ef3ff50dc5c7ff71345


KeyboardInterrupt: 

In [ ]:
import pickle

with open('./train_meta_dict.pickle', 'wb') as handle:
    pickle.dump(train_meta_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)